## Proyecto Módulo 2 - Modelamiento Estadístico
### Maria Fernanda Palacio, Alejandro Vega y Juan Sebastian Rodriguez

### Dependencias

In [2]:
from utils import *
import pandas as pd
import numpy as np
import seaborn as sns
import tomotopy as tp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import torch
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from sklearn.preprocessing import FunctionTransformer
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

### Dataset

In [3]:
df_movies = pd.read_csv('movies_metadata.csv',low_memory=False)
df_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


### Preprocesamiento de datos

In [4]:
df = preprocesamiento_datos(df_movies)

In [5]:
df['genre'].value_counts()

genre
Drama              4909
Comedy             3371
Documentary        2685
Horror              971
Thriller            455
Western             317
Action              276
Animation           237
Science Fiction     195
Crime               127
Music               109
Adventure           107
Name: count, dtype: int64

Se aplica un preprocesamiento inicial al dataset. La distribución de géneros muestra que "Drama" es el más común con 4909 películas, seguido por "Comedy" con 3371 y "Documentary" con 2685 películas. Otros géneros tienen menor representación.

### Fragmentación de sinopsis

In [6]:
df_valid = codificar_genero(df)

df_valid[["genre", "genre_code"]].head()

Diccionario género → código: {'Action': 0, 'Adventure': 1, 'Animation': 2, 'Comedy': 3, 'Crime': 4, 'Documentary': 5, 'Drama': 6, 'Horror': 7, 'Music': 8, 'Science Fiction': 9, 'Thriller': 10, 'Western': 11}


,genre,genre_code
4,Comedy,3
25,Drama,6
35,Drama,6
39,Drama,6
54,Drama,6


Codificamos los géneros con valores numéricos para facilitar el análisis, el resultado muestra el mapeo de cada género a un código numérico.

In [7]:
splitter = SentenceTransformersTokenTextSplitter(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        tokens_per_chunk=256,
        chunk_overlap=0
)

mask_one_chunk = df_valid["overview"].apply(
        lambda txt: len(splitter.split_text(str(txt))) == 1
)
df_chunk = df_valid[mask_one_chunk].copy()

print(f"Después del paso 5 (sinopsis de un solo bloque): {df_chunk.shape}")

c:\Users\alejo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Después del paso 5 (sinopsis de un solo bloque): (13745, 26)


Utilizamos un tokenizador para dividir las sinopsis, manteniendo solo aquellas que caben en un bloque de 256 tokens. Esto reduce nuestro dataset a 13,745 películas.

In [8]:
df_chunk['label'] = df_chunk['genre'].apply(lambda x: 1 if x == 'Drama' else 0)
df_final = df_chunk[["title", "overview", "genre", "label"]].copy()

print(f"Después del paso 6 (columnas reducidas): {df_final.shape}")
df_final.head(2)

Después del paso 6 (columnas reducidas): (13745, 4)


,title,overview,genre,label
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,0
25,Othello,The evil Iago pretends to be friend of Othello...,Drama,1


Creamos una variable objetivo binaria donde 1 indica que la película es del género "Drama" y 0 que pertenece a otro género, y reducimos el dataset a las columnas relevantes.

### Tokenización

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 1. Convertir a minúsculas
    text = text.lower()
    # 2. Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 3. Tokenizar
    tokens = text.split()
    # 4. Eliminar stopwords y lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

df_final['processed_text'] = df_final['overview'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alejo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alejo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Implementamos un proceso de tokenización que incluye:

- Conversión a minúsculas
- Eliminación de caracteres especiales
- Tokenización del texto
- Eliminación de palabras vacías
- Lematización para reducir palabras a su forma base

In [10]:
df_final.head()

,title,overview,genre,label,processed_text
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,0,"[george, bank, recovered, daughter, wedding, r..."
25,Othello,The evil Iago pretends to be friend of Othello...,Drama,1,"[evil, iago, pretend, friend, othello, order, ..."
35,Dead Man Walking,A justice drama based on a true story about a ...,Drama,1,"[justice, drama, based, true, story, man, deat..."
39,"Cry, the Beloved Country",A South-African preacher goes to search for hi...,Drama,1,"[southafrican, preacher, go, search, wayward, ..."
54,Georgia,"Sadie looks up to her older sister Georgia, a ...",Drama,1,"[sadie, look, older, sister, georgia, successf..."


### Determinar el número de temas óptimo (p)

In [11]:
import tomotopy as tp
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

def find_optimal_topics(docs, target_genre, genre_labels, min_topics=10, max_topics=20, k_folds=5):

    binary_labels = np.array([1 if g == target_genre else 0 for g in genre_labels])

    results = {}

    for n_topics in range(min_topics, max_topics + 1):
        f1_scores = []
        skf = StratifiedKFold(n_splits=k_folds)

        for train_idx, test_idx in skf.split(docs, binary_labels):
            # Entrenar modelo sLDA
            mdl = tp.SLDAModel(k=n_topics, vars='b')
            for i in train_idx:
                mdl.add_doc(docs[i], y=[binary_labels[i]])
            mdl.train(iter=500)

            # Predecir en fold de prueba
            preds = []
            for i in test_idx:
                doc = mdl.make_doc(docs[i])
                pred = mdl.infer(doc)[0]
                preds.append(1 if pred[1] > 0.5 else 0)

            # Calcular F1-score
            f1_scores.append(f1_score(binary_labels[test_idx], preds, zero_division=0))

        results[n_topics] = np.mean(f1_scores)

    optimal_n = max(results.items(), key=lambda x: x[1])[0]
    return results, optimal_n

Definimos una función para encontrar el número óptimo de temas usando validación cruzada, además, la función evalúa modelos con diferentes cantidades de temas y selecciona el que maximice el F1-score.

In [12]:
target_genre = 'Drama'
results, optimal_topics = find_optimal_topics(
    docs=df_final['processed_text'].values,
    target_genre=target_genre,
    genre_labels=df_final['genre'].values
)

print(f"Número óptimo de temas: {optimal_topics}")
print("Resultados por número de temas:", results)

Número óptimo de temas: 10
Resultados por número de temas: {10: 0.010556207089568314, 11: 0.005649687466078442, 12: 0.001599450064145861, 13: 0.0012121212121212121, 14: 0.0008113723796004856, 15: 0.0012137570032306874, 16: 0.002026350797566732, 17: 0.0024120603015075374, 18: 0.0, 19: 0.003631100387453402, 20: 0.0}


Los resultados indican que 10 temas es la configuración óptima, aunque los F1-scores son bajos en general, esto sugiere limitaciones en el enfoque de modelado de tópicos para esta tarea específica.

### Entrenamiento del modelo

In [13]:
documentos = df_final['processed_text'].tolist()
labels = df_final['label'].tolist()

In [ ]:
modelo_binario = tp.SLDAModel(k=10, seed=123,vars='b')
#for i in range(len(X_train)):
#    modelo_binario.add_doc(words = X_train[i], y=[X_test[i]])
for tokens, label in zip(documentos, labels):
    modelo_binario.add_doc(tokens, y=[label])
# Entrenar el modelo
modelo_binario.train(100)

modelo_binario.save("slda_model.bin")

C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3647714078.py:7: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  modelo_binario.train(100)


Entrenamos el modelo sLDA con 10 temas utilizando nuestro dataset procesado y guardamos el modelo para uso futuro.

### Interpretación de los temas

In [15]:
print("\nTemas Aprendidos:")
for k in range(modelo_binario.k):
    print(f"Tema #{k}: {modelo_binario.get_topic_words(k,top_n=10)}")


Temas Aprendidos:
Tema #0: [('family', 0.012981079518795013), ('find', 0.012165524996817112), ('young', 0.011944646015763283), ('wife', 0.011927654966711998), ('father', 0.011706775985658169), ('woman', 0.011689784936606884), ('home', 0.010041684843599796), ('son', 0.008580483496189117), ('man', 0.00822367798537016), ('daughter', 0.00800279900431633)]
Tema #1: [('police', 0.010264166630804539), ('crime', 0.007931471802294254), ('team', 0.00755824102088809), ('get', 0.007216112222522497), ('murder', 0.00715390732511878), ('two', 0.007091701962053776), ('found', 0.00690508633852005), ('drug', 0.006562958005815744), ('case', 0.005785393062978983), ('go', 0.005443264730274677)]
Tema #2: [('film', 0.05136427283287048), ('documentary', 0.01979593001306057), ('movie', 0.009898066520690918), ('director', 0.009595687501132488), ('story', 0.007862049154937267), ('filmmaker', 0.006592058576643467), ('first', 0.006551741622388363), ('history', 0.006168728228658438), ('world', 0.006108252797275782

Los 10 temas identificados por el modelo muestran patrones semánticos interesantes:

- Tema 0: Centrado en relaciones familiares (palabras como "family", "father", "wife")
- Tema 1: Relacionado con crimen y policía
- Tema 2: Enfocado en documentales y cine
- Tema 3: Sobre relaciones y experiencias personales
- Tema 4: Ubicaciones y personajes específicos
- Tema 5: Experiencias cotidianas y amistad
- Tema 6: Narrativas y viajes
- Tema 7: Temas políticos y bélicos
- Tema 8: Escenarios de misterio y suspenso
- Tema 9: Entretenimiento y comedia


### Creación de 3 reviews ficticias

In [16]:
def test_data():
  paragraphs = [

    {"overview": "Cuando una abuela viral en redes sociales se muda con su nieto programador, el caos doméstico y los malentendidos digitales desatan una ola de risas, memes y momentos inolvidables.", "genre":"Comedy","label": 0, "title": "Cosas de Familia"},
    {"overview": "Tras la muerte de su padre, Elena encuentra una carta sin abrir que revela un amor prohibido. En su búsqueda por descubrir la verdad, enfrentará recuerdos dolorosos, secretos familiares y la posibilidad de una nueva vida.", "genre":"Drama","label": 1, "title": " La Última Carta"},
    {"overview": "Después de una tragedia que sacudió a su familia, Julián regresa a su pueblo natal para cuidar a su hermano menor. Enfrentado al peso del pasado y a viejas heridas, deberá reconciliarse con lo que fue y decidir quién quiere ser.", "genre":"Drama","label": 1, "title": "Ecos del Silencio"},
  ]
  return pd.DataFrame(paragraphs)


Creamos tres sinopsis ficticias para probar nuestro modelo: una comedia y dos dramas.

In [17]:
df_test = test_data()
df_test.head()

,overview,genre,label,title
0,Cuando una abuela viral en redes sociales se m...,Comedy,0,Cosas de Familia
1,"Tras la muerte de su padre, Elena encuentra un...",Drama,1,La Última Carta
2,Después de una tragedia que sacudió a su famil...,Drama,1,Ecos del Silencio


In [18]:
df_test['processed_text'] = df_test['overview'].apply(preprocess_text)

In [19]:
doc = modelo_binario.make_doc(df_test['overview'][1])
pred_prob = modelo_binario.infer(doc)[0][1]  # Probabilidad de ser del género objetivo

print(f"Probabilidad de ser '{target_genre}': {pred_prob:.4f}")

Probabilidad de ser 'Drama': 0.0524


C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\982813350.py:1: RuntimeWarning: `words` should be an iterable of str.
  doc = modelo_binario.make_doc(df_test['overview'][1])


Al evaluar una de las sinopsis de drama, el modelo asigna una probabilidad muy baja (0.0524) de que pertenezca al género Drama, indicando un mal desempeño del modelo.

### Evaluación del error del modelo mediante validación cruzada (k=10) usando ROC-AUC

In [20]:
X = df_final['processed_text'].tolist()
y = df_final['label'].tolist()

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
auc_scores = []

for train_idx, test_idx in kf.split(X, y):
    X_train = [X[i] for i in train_idx]
    y_train = [y[i] for i in train_idx]
    X_test = [X[i] for i in test_idx]
    y_test = [y[i] for i in test_idx]

    model = tp.SLDAModel(k=10, seed=123, vars='b')
    for tokens, label in zip(X_train, y_train):
        model.add_doc(tokens, y=[label])
    model.train(100)

    y_pred_prob = []
    for tokens in X_test:
        doc = model.make_doc(tokens)
        prob = model.infer(doc)[0][1]
        y_pred_prob.append(prob)
    auc = roc_auc_score(y_test, y_pred_prob)
    auc_scores.append(auc)

print(f"ROC-AUC promedio (10 folds): {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")

C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
C:\Users\alejo\AppData\Local\Temp\ipykernel_11168\3893411064.py:16: RuntimeWarning: The training result may differ even with fixed

ROC-AUC promedio (10 folds): 0.4721 ± 0.0497


Realizamos una evaluación rigurosa usando validación cruzada de 10 folds. El ROC-AUC promedio de 0.4721 ± 0.0497 es inferior a 0.5 (equivalente al azar), confirmando que el modelo no es efectivo para clasificar géneros basándose solo en los tópicos de las sinopsis.

### Vector de regresión

In [28]:
print("Vector de regresión (coeficientes de cada tema para la variable binaria):")
print(modelo_binario.get_regression_coef())

Vector de regresión (coeficientes de cada tema para la variable binaria):
[[ 2.1392384e+00 -1.4350755e+00 -3.0051587e+00  5.7812428e+00
  -3.9237449e+00 -1.9942586e+00 -2.4438375e-01 -1.2806170e-03
  -4.3325233e+00 -3.6554732e+00]]


El vector de regresión muestra cómo cada tema contribuye a la clasificación:

- El tema 3 tiene el coeficiente más alto (5.78), indicando fuerte asociación con el género Drama
- Los temas 8, 4 y 9 tienen coeficientes muy negativos, sugiriendo que están asociados con géneros no-Drama

Estos coeficientes ofrecen información sobre qué temas son característicos del género Drama frente a otros géneros, aunque el modelo en general no tenga un buen desempeño predictivo.